## Lesson 1: Router Engine
https://learn.deeplearning.ai/courses/building-agentic-rag-with-llamaindex/lesson/2/router-query-engine

In [20]:
import os

from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv())

OPENAI_MODEL = os.getenv("OPENAI_MODEL")

EMBEDDING_API_KEY = os.getenv("EMBEDDING_API_KEY")

In [2]:
# Make asyncio loops work inside Jupyter notebooks which already have loops under the hood
import nest_asyncio

nest_asyncio.apply()

In [4]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(input_files=["./metagpt.pdf"]).load_data()

In [5]:
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=1024)

nodes = splitter.get_nodes_from_documents(documents=documents)

#### Define LLM and Embedding model

In [69]:
from llama_index.core import Settings
from llama_index.embeddings.jinaai import JinaEmbedding
from llama_index.llms.openai_like import OpenAILike

Settings.embed_model = JinaEmbedding(api_key=EMBEDDING_API_KEY)
Settings.llm = OpenAILike(model=OPENAI_MODEL, temperature=0.01, is_chat_model=True)

#### Define Summary Index and Vector Index over the Same Data

In [70]:
from llama_index.core import SummaryIndex, VectorStoreIndex

summary_index = SummaryIndex(nodes=nodes)

vector_index = VectorStoreIndex(nodes=nodes)

#### Define Query Engines and Set Metadata

In [71]:
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize", use_async=True
)

vector_query_engine = vector_index.as_query_engine()

In [72]:
from llama_index.core.tools import QueryEngineTool

summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description="Useful for summarization questions regarding MetaGPT",
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description="Useful for retrieving specific context from MetaGPT paper",
)

#### Define Router Query Engine

In [73]:
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector

query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[summary_tool, vector_tool],
    verbose=True,
)

In [74]:
response = query_engine.query("What is the summary of the document?")

print(response)

Selecting query engine 0: The question 'What is the summary of the document?' is general and asks for a summarization, which corresponds to choice 1 that is described as useful for summarization questions..
The document provides an overview of MetaGPT, a meta-programming framework designed to enhance the capabilities of LLM-based multi-agent systems. It details how MetaGPT incorporates structured communication, role-based collaboration, and efficient information sharing mechanisms to improve code generation and development workflows. The framework's effectiveness is demonstrated through benchmarks and experimental results, showcasing its superiority over previous systems. It also discusses the importance of different roles, an executable feedback mechanism for iterative code improvement, and the overall potential of human-inspired techniques in artificial multi-agent systems. Additionally, the document presents performance metrics, issues, and fixes associated with various versions or 

In [75]:
print(len(response.source_nodes))

34


In [76]:
response = query_engine.query("How do agents share information with other agents?")

print(response)

Selecting query engine 1: The question asks about a specific mechanism or process, which is likely described in the context of the MetaGPT paper. Therefore, choice 2, which is about retrieving specific context from the paper, is more relevant..
Agents share information with other agents by using a shared message pool where they can publish structured messages. They can also subscribe to relevant messages based on their profiles, which allows for efficient role communication within the system.


In [77]:
response = query_engine.query("Tell me about the ablation study results?")

print(response)

Selecting query engine 1: The question about ablation study results is specific to the details provided in the MetaGPT paper, which requires retrieving specific context from the paper..
The ablation study results are not explicitly mentioned in the provided context. However, based on the information given, we can infer that the study involves comparing the performance of MetaGPT without feedback (w/o Feedback) on the SoftwareDev dataset. The results are presented in Table 9, which includes metrics such as code statistics, document statistics, cost statistics, and code executability for 10 randomly selected tasks. The average values of these metrics are also provided, indicating the performance of MetaGPT in terms of code quality, cost, and executability. Unfortunately, specific details about the ablation study, such as the comparison with other variations of MetaGPT or the impact of different components, are not mentioned in the context.
